In [3]:
import Bio
import os
import pandas as pd
from subprocess import call

In [ ]:
for dirpath, dirnames, files in os.walk('../'):
    for file_name in files:
        file_path = os.path.join(dirpath, file_name)
        if file_name == 'fasta_file.fasta':
            blastdb = 'makeblastdb -in '+file_path+' -out '+dirpath+'/fastas_neoprospecta -dbtype nucl'
            call(blastdb, shell=True)

In [ ]:
for dirpath, dirnames, files in os.walk('../'):
    for file_name in files:
        file_path = os.path.join(dirpath, file_name)
        if file_name.endswith('001.fastq'):
            file_id = file_name.split('_')[0]
            
            os.mkdir('fqs/'+file_id+')
            
            fastqc_pre_trimm = 'fastqc '+file_path+''
            
            trimmomatic = 'trimmomatic SE -threads 20 '+file_path+' fqs/'+file_id+'/'+file_id+'_R1_filt1.fastq LEADING:5 TRAILING:5 SLIDINGWINDOW:4:20 CROP:245 MINLEN:243'
            
            fastqc_pos_trimm = 'fastqc fqs/'+file_id+'_R1_filt1.fastq'
            
            fq2fa = 'fq2fa fqs/'+file_id+'/'+file_id+'_R1_filt1.fastq fqs/'+file_id+'/'+file_id+'_R1_filt1.fasta'
            
            blastn = 'blastn -db /media/labim/HDD/Documentos/boi/Desafio_Neoprospecta/database/fastas_neoprospecta -query fqs/'+file_id+'/'+file_id+'_R1_filt1.fasta -out fqs/'+file_id+'/result_blast00_'+file_id+'.csv -outfmt "10 qacc sacc pident length mismatch evalue" -num_threads 10 -max_target_seqs 1'
            
            call(fastqc, shell=True)
            call(trimmomatic, shell=True)
            call(fastqc_pos_trimm, shell=True)
            call(fq2fa, shell=True)
            call(blastn, shell=True)

In [70]:
with open('tables/tax_table_amostras.tsv') as taxon_table:
    df_taxons = pd.read_csv(taxon_table, sep='\t')
    df_taxons['nomes'] = ""
    df_taxons['codigos'] = ""
        
    for names in df_taxons['Unnamed: 0']:
        subj_acc = names.split(' ')[0]
        names_ = str(names).replace('[','').replace(']','').strip(subj_acc)        
        df_taxons.loc[df_taxons['Unnamed: 0'] == names, 'nomes'] = names_
        df_taxons.loc[df_taxons['Unnamed: 0'] == names, 'codigos'] = int(subj_acc)
        df_taxons['codigos'] = pd.to_numeric(df_taxons['codigos'])
    corte_tabela = df_taxons[['nomes','codigos']]


for dirpath, dirnames, files in os.walk('../'):
    for file_name in files:
        file_path = os.path.join(dirpath, file_name)
        if file_name.startswith('result_blast00'):
            file_id = file_path.split('/')[3]
            
            df1 = pd.read_csv(file_path, header=None)
            df1.rename(columns={0:'qacc',1:'codigos'}, inplace = True)
            
            corte_tabela2 = df1[['qacc','codigos']]
            
            df3 = pd.merge(corte_tabela, corte_tabela2, how = 'inner', on = 'codigos')
            df3.to_csv('fqs/'+file_id+'/merged_results_'+file_id+'.csv', index=False)

In [ ]:
for dirpath, dirnames, files in os.walk('../'):
    for file_name in files:
        file_path = os.path.join(dirpath, file_name)
        if file_name.startswith('merged_results'):
            file_id = file_path.split('/')[3]
            contagens00 = pd.read_csv(file_path)
            values = contagens00['nomes'].value_counts().keys().tolist()
            counts = contagens00['nomes'].value_counts().tolist()
            
            micros = {}
            for keys in values:
                for count in counts:
                    micros[keys] = count
                    counts.remove(count)
                    break

            Micros_counts = pd.DataFrame([micros])
            
            micros_counts_inver = Micros_counts.T
            micros_counts_inver.rename(columns={0:file_id}, inplace = True)
            
            micros_counts_inver.to_csv('fqs/'+file_id+'/count_results_'+file_id+'.csv')


In [133]:
files_paths = []

for dirpath, dirnames, files in os.walk('./'):
    for file_name in files:
        file_path = os.path.join(dirpath, file_name)
        if file_name.startswith('count') and file_name.endswith('.csv'):
            files_paths.append(file_path)

taxon_count_total = pd.concat(map(pd.read_csv, files_paths), axis=0, join='outer')
taxon_count_total = taxon_count_total.fillna(0)
taxon_count_total.sort_index(axis=1, inplace=True)
taxon_count_total = taxon_count_total.groupby('Unnamed: 0').sum().to_csv('Total_count_final.csv', header=True)
